In [25]:
import io
import xml.etree.ElementTree as ET
import pandas as pd
import helper
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import numpy as np
import os
#from gensim import corpora, models, similarities 

time: 2.56 ms


In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 517 ms


In [27]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 778 ms


In [28]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

time: 1.12 ms


[autoreload of autotime failed: Traceback (most recent call last):
  File "/home/jshoun01/.virtualenvs/py27/local/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named autotime
]


In [29]:
ls data

data@
time: 860 ms


In [30]:
data_dir='data'

time: 186 ms


In [31]:

posts_df = helper.get_xml_file_as_df(os.path.join(data_dir,'Posts.xml'),max_num_children=5000)

#posts_df = helper.get_xml_file_as_df(os.path.join(data_dir,'Posts.xml'))

time: 52.8 s


In [32]:
posts_df.columns

Index([u'AcceptedAnswerId', u'AnswerCount', u'Body', u'ClosedDate',
       u'CommentCount', u'CommunityOwnedDate', u'CreationDate',
       u'FavoriteCount', u'Id', u'LastActivityDate', u'LastEditDate',
       u'LastEditorDisplayName', u'LastEditorUserId', u'OwnerDisplayName',
       u'OwnerUserId', u'ParentId', u'PostTypeId', u'Score', u'Tags', u'Title',
       u'ViewCount'],
      dtype='object')

time: 2.42 ms


In [33]:
posts_df.irow(0).Body

u'<p>A Vista virtual machine I use only has a 10 GB virtual hard disk -- and I\'m worried about it running out of space over time.</p>\n\n<p>I used <a href="http://www.jam-software.com/treesize_free/">TreeSize</a> to check for the culprits..</p>\n\n<p><img src="http://i.stack.imgur.com/uGfhQ.png" alt="enter image description here"></p>\n\n<p>And the primary culprit is that <strong><code>/winsxs</code> or "Windows Side-By-Side" folder</strong>.</p>\n\n<p><img src="http://i.stack.imgur.com/DWYOo.png" alt="enter image description here"></p>\n\n<p>What is the Windows Side By Side folder? Well, that\'s <a href="http://blogs.technet.com/askcore/archive/2008/09/17/what-is-the-winsxs-directory-in-windows-2008-and-windows-vista-and-why-is-it-so-large.aspx">explained pretty well here</a>.</p>\n\n<blockquote>\n  <p>All of the components in the operating system are found in the WinSxS folder \u2013 in fact we call this location the component store.  Each component has a unique name that includes t

time: 710 ms


In [34]:
posts_df.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorDisplayName,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
0,727273,10,<p>A Vista virtual machine I use only has a 10...,NaN,9,NaN,2009-07-15T06:27:46.723,69,1,2014-10-18T07:23:25.843,...,NaN,73637,NaN,1,NaN,1,132,<windows><disk-space><winsxs>,"Why does the /winsxs folder grow so large, and...",70584
1,NaN,6,<p>I used to record important technical meetin...,NaN,0,NaN,2009-07-08T16:32:47.817,3,24,2012-08-08T21:06:48.087,...,NaN,36744,essamSALAH,NaN,NaN,1,5,<audio><audio-recording><camtasia>,How can I remove noise from an audio recording?,3654
2,NaN,NaN,<p>You could use a filter on the recording. If...,NaN,0,NaN,2009-07-08T16:34:52.867,NaN,25,2009-07-15T06:48:44.300,...,NaN,NaN,Burkhard,9637,24,2,0,NaN,NaN,NaN
3,NaN,NaN,<p>try out something like:</p>\n\n<p>audible (...,NaN,0,NaN,2009-07-08T16:59:00.587,NaN,26,2009-07-15T06:48:44.300,...,NaN,NaN,Jas Panesar,23242,24,2,0,NaN,NaN,NaN
4,NaN,NaN,<p>You definitely need some sort of software t...,NaN,1,NaN,2009-07-08T17:15:59.477,NaN,27,2009-07-15T06:48:44.300,...,NaN,NaN,T Pops,45675,24,2,0,NaN,NaN,NaN


time: 571 ms


Only interested in questions

In [35]:
print("There are %s posts"%(len(posts_df)))
posts_df=posts_df[posts_df.PostTypeId=="1"]
print("There are %s question posts"%(len(posts_df)))

There are 5001 posts
There are 1293 question posts
time: 257 ms


## Process Text

In [36]:
valid_pos= ["FW","NN","NNS","NNP","NNPS","VB","VBZ","VBP","VBD","VBN","VBG"]

vali_pos_set=set(valid_pos)

time: 336 ms


In [48]:
#pos tagging
posts_df['BodyCleaned']=posts_df['Body'].apply(lambda x : helper.filter_parts_of_speech(x,vali_pos_set))

# strip html tags
posts_df['BodyCleaned']=posts_df['BodyCleaned'].apply(lambda x : helper.strip_tags(x))
# tokenize text
posts_df['BodyCleaned']=posts_df['BodyCleaned'].apply(lambda x : helper.tokenize_and_stem(x,None))

# remove stop words
posts_df['BodyCleaned']=posts_df['BodyCleaned'].apply(lambda x : helper.remove_stopwords(x))



time: 3.21 s


In [49]:
posts_df.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount,BodyCleaned
0,727273,10,<p>A Vista virtual machine I use only has a 10...,NaN,9,NaN,2009-07-15T06:27:46.723,69,1,2014-10-18T07:23:25.843,...,73637,NaN,1,NaN,1,132,<windows><disk-space><winsxs>,"Why does the /winsxs folder grow so large, and...",70584,"[A, Vista, machine, use, GB, disk, m, worried,..."
1,NaN,6,<p>I used to record important technical meetin...,NaN,0,NaN,2009-07-08T16:32:47.817,3,24,2012-08-08T21:06:48.087,...,36744,essamSALAH,NaN,NaN,1,5,<audio><audio-recording><camtasia>,How can I remove noise from an audio recording?,3654,"[I, used, record, meetings, demos, using, Camt..."
5,42,9,<p>What is the difference between the two supe...,NaN,0,NaN,2009-07-15T07:11:46.370,1,29,2011-11-25T19:11:50.960,...,48078,NaN,13,NaN,1,17,<linux><unix><sudo>,"What's the difference between the commands ""su...",14700,"[What, difference, user, commands, su, sudo, T..."
7,3841,3,"<p>My late 2008, Macbook Pro Unibody machine h...",NaN,2,NaN,2009-07-15T07:17:13.970,1,37,2009-07-15T21:15:21.323,...,NaN,NaN,26,NaN,1,4,<mac><crash><boot-camp>,Why does the Macbook Pro Unibody crash on hibe...,1169,"[My, Macbook, Pro, Unibody, machine, number, f..."
13,92,2,<p>Is there any way to configure application t...,NaN,0,NaN,2009-07-15T07:20:00.717,1,46,2014-07-26T13:37:09.967,...,81403,NaN,21,NaN,1,9,<windows-vista><boot><uac><privileges><elevation>,Elevated Priviliges for Startup Applications i...,7997,"[Is, way, configure, application, requires, ad..."


time: 33.5 ms


In [50]:
posts_df.irow(0)['Body']

u'<p>A Vista virtual machine I use only has a 10 GB virtual hard disk -- and I\'m worried about it running out of space over time.</p>\n\n<p>I used <a href="http://www.jam-software.com/treesize_free/">TreeSize</a> to check for the culprits..</p>\n\n<p><img src="http://i.stack.imgur.com/uGfhQ.png" alt="enter image description here"></p>\n\n<p>And the primary culprit is that <strong><code>/winsxs</code> or "Windows Side-By-Side" folder</strong>.</p>\n\n<p><img src="http://i.stack.imgur.com/DWYOo.png" alt="enter image description here"></p>\n\n<p>What is the Windows Side By Side folder? Well, that\'s <a href="http://blogs.technet.com/askcore/archive/2008/09/17/what-is-the-winsxs-directory-in-windows-2008-and-windows-vista-and-why-is-it-so-large.aspx">explained pretty well here</a>.</p>\n\n<blockquote>\n  <p>All of the components in the operating system are found in the WinSxS folder \u2013 in fact we call this location the component store.  Each component has a unique name that includes t

time: 575 ms


In [51]:
posts_df.irow(0)['BodyCleaned']

[u'A',
 u'Vista',
 u'machine',
 u'use',
 u'GB',
 u'disk',
 u'm',
 u'worried',
 u'running',
 u'space',
 u'time',
 u'I',
 u'used',
 u'TreeSize',
 u'check',
 u'culprits..',
 u'And',
 u'culprit',
 u'/winsxs',
 u'Windows',
 u'Side-By-Side',
 u'folder',
 u'What',
 u'Windows',
 u'Side',
 u'Side',
 u'folder',
 u'explained',
 u'All',
 u'components',
 u'operating',
 u'system',
 u'found',
 u'WinSxS',
 u'folder',
 u'fact',
 u'call',
 u'location',
 u'component',
 u'store',
 u'component',
 u'name',
 u'includes',
 u'version',
 u'language',
 u'processor',
 u'architecture',
 u'built',
 u'WinSxS',
 u'folder',
 u'location',
 u'component',
 u'found',
 u'system',
 u'instances',
 u'files',
 u'see',
 u'system',
 u'projected',
 u'linking',
 u'component',
 u'store',
 u'Let',
 u'repeat',
 u'point',
 u'instance',
 u'data',
 u'copy',
 u'version',
 u'file',
 u'OS',
 u'instance',
 u'located',
 u'WinSxS',
 u'folder',
 u'looked',
 u'perspective',
 u'WinSxS',
 u'folder',
 u'entirety',
 u'OS',
 u'referred',
 u'operatin

time: 558 ms


In [52]:
len(posts_df)

1293

time: 567 ms


In [53]:
texts = posts_df['BodyCleaned']

time: 628 ms


In [54]:
#create a Gensim dictionary from the texts
dictionary = gensim.corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

time: 478 ms


In [55]:
corpus[0][:10]

[(51, 1),
 (72, 1),
 (78, 1),
 (122, 1),
 (128, 2),
 (135, 2),
 (299, 1),
 (304, 1),
 (318, 1),
 (443, 1)]

time: 423 ms


In [56]:
#gensim.models.LdaMulticore
lda = gensim.models.LdaMulticore(corpus, num_topics=10, passes=10,
                            id2word=dictionary)

time: 40.1 s


In [57]:
lda.show_topics(num_words=20)


[u'0.025*I + 0.018*drive + 0.018*n + 0.017*m + 0.013*use + 0.008*want + 0.008*Windows + 0.008*Is + 0.008*computer + 0.007*ve + 0.007*something + 0.006*file + 0.006*get + 0.006*looking + 0.005*way + 0.005*using + 0.004*window + 0.004*time + 0.004*d + 0.004*know',
 u'0.021*r + 0.021*None + 0.016*I + 0.012*Windows + 0.010*y + 0.010*dc + 0.010*tc + 0.008*n + 0.007*Is + 0.006*ve + 0.005*work + 0.005*computer + 0.005*m + 0.005*using + 0.004*know + 0.004*get + 0.004*OS + 0.004*Firefox + 0.004*use + 0.004*version',
 u'0.020*I + 0.013*use + 0.008*m + 0.008*Windows + 0.007*monitor + 0.007*n + 0.006*ve + 0.006*work + 0.005*How + 0.005*using + 0.005*get + 0.005*laptop + 0.005*d + 0.005*XP + 0.005*know + 0.004*anyone + 0.004*time + 0.004*keyboard + 0.004*OS + 0.004*windows',
 u'0.019*I + 0.011*n + 0.011*get + 0.008*m + 0.007*iTunes + 0.006*ve + 0.006*way + 0.006*power + 0.005*work + 0.005*Is + 0.005*iPhone + 0.004*problem + 0.004*time + 0.004*keyboard + 0.004*using + 0.004*got + 0.004*What + 0.004*

time: 5.12 ms


In [58]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

['I', 'drive', 'n', 'm', 'use', 'want', 'Windows', 'Is', 'computer', 've', 'something', 'file', 'get', 'looking', 'way', 'using', 'window', 'time', 'd', 'know']
()
['r', 'None', 'I', 'Windows', 'y', 'dc', 'tc', 'n', 'Is', 've', 'work', 'computer', 'm', 'using', 'know', 'get', 'OS', 'Firefox', 'use', 'version']
()
['I', 'use', 'm', 'Windows', 'monitor', 'n', 've', 'work', 'How', 'using', 'get', 'laptop', 'd', 'XP', 'know', 'anyone', 'time', 'keyboard', 'OS', 'windows']
()
['I', 'n', 'get', 'm', 'iTunes', 've', 'way', 'power', 'work', 'Is', 'iPhone', 'problem', 'time', 'keyboard', 'using', 'got', 'What', 'find', 'know', 'system']
()
['I', 'n', 'Windows', 'video', 'Vista', 've', 'card', 'laptop', 'm', 'work', 'running', 'monitor', 'The', 'problem', 'computer', 'files', 'using', 'Is', 'XP', 'way']
()
['I', 'Windows', 'n', 'network', 'PM', 'm', 'Vista', 'XP', 'computer', 'Is', 'using', 'use', 'rwxp', 'system', 've', 'want', 'time', 'running', 'Ubuntu', 'way']
()
['Flash', 'I', 'cookies', 'P